In [1]:
# Usuals imports 
import numpy as np
import pandas as pd 
import requests

## Cities 

In [2]:
cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

## Dataframe Cities 

In [3]:
cities_locations = pd.DataFrame(columns= ['city', 'country', 'latitude', 'longitude', 'adress'])

for city in cities:
    country = 'France'
    latitude = 0.0
    longitude = 0.0
    adress = ''

    # Creating urls params
    payload = {'city': city, 'country': country, 'format': 'jsonv2'}

    # Create the requests 
    r = requests.get('https://nominatim.openstreetmap.org/search.php?', params=payload)

    # check result available 
    if r.status_code == 200 :
        r_result = pd.DataFrame(r.json())
        #attrib location 
        try: 
            latitude = r_result['lat'][0]
            longitude = r_result['lon'][0]
            adress =  r_result['display_name'][0]

        except:
            #try with county 
            payload = {'county': city, 'country': country, 'format': 'jsonv2'}
            r = requests.get('https://nominatim.openstreetmap.org/search.php?', params=payload)
            r_result = pd.DataFrame(r.json())

            try: 
                latitude = r_result['lat'][0]
                longitude = r_result['lon'][0]
                adress =  r_result['display_name'][0]

            except:
                #try with street 
                payload = {'street': city, 'country': country, 'format': 'jsonv2'}
                r = requests.get('https://nominatim.openstreetmap.org/search.php?', params=payload)
                r_result = pd.DataFrame(r.json())

                try: 
                    latitude = r_result['lat'][0]
                    longitude = r_result['lon'][0]
                    adress =  r_result['display_name'][0]
                
                except:
                    pass

        new_city = pd.DataFrame(columns= ['city', 'country', 'latitude', 'longitude', 'adress'], 
                                   data= [[ city, country, latitude, longitude, adress]])

        cities_locations = pd.concat([cities_locations, new_city])

# reset index due to concat.
cities_locations.reset_index(inplace= True)
cities_locations.drop(columns=['index'], inplace= True)

## Dataframe Wheater in Cities :

In [4]:
# https://openweathermap.org/api/one-call-3
# 
KEYS = open('src/KEYS.txt', 'r').read()

latitude = cities_locations['latitude'][0]
longitude = cities_locations['longitude'][0]

# Creating urls params
payload = {'lat': latitude, 'lon': longitude, 'appid': KEYS}

# Create the requests 
r = requests.get('https://api.openweathermap.org/data/3.0/onecall?', params=payload)

print(r.status_code)

print(r.content)


401
b'{"cod":401, "message": "Invalid API key. Please see http://openweathermap.org/faq#error401 for more info."}'


## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse